![](https://axisimagingnews.com/wp-content/uploads/2020/10/COVID19.jpg)

Source: https://axisimagingnews.com/imaging-news/associations/siim-fisabio-rsna-host-machine-learning-challenge-covid-19-detection-localization


# The CoVID-19 crisis 
In the past year has affected the world. 

Currently, COVID-19 can be diagnosed, it can take a few hours and sometimes days before results are back.

So I try to make guidline to help radiologist differentiate COVID-19 from other types of infection.

Let's drive right in!


#### Study-level labels

Studies in the test set contain labels so They are as follows:
        
            "negative", "typical", "indeterminate", "atypical"



### Content

**NEGATIVE FOR PNEUMONIA**

    - No lung opacities

**TYPICAL APPEARANCE**

    - Multifocal bilateral, peripheral opacities with rounded morphology, lower lung–predominant distribution

**INDETERMINATE APPEARANCE**

    - Absence of typical findings AND unilateral, central or upper lung predominant distribution

**ATYPICAL APPEARANCE**

    - Pneumothorax, pleural effusion, pulmonary edema, lobar consolidation, solitary lung nodule or mass, diffuse tiny nodules, cavity

##### 1. Import Libraries
##### 2. Collect data
##### 3. Review
##### 4. Data Exploration (label group)

      * Negative for Pneumonia

      * Typical Appearance

      * Indeterminate Appearance

      * Atypical_Appearance
 ##### 5. Data Exploation (Photometric Interpretation and Modality types.)
 
      * MONOCROME1 & CR

      * MONOCROME1 & DX

      * MONOCROME2 & CR

      * MONOCROME2 & DX

 ##### 6. Ranking of Gray of MONOCROME1 & CR
 
 

        

## 1. Import Libraries

In [ ]:
!pip install python-gdcm -q

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from matplotlib.gridspec import GridSpec
%matplotlib inline

from functools import reduce
import ast
import seaborn as sns

from IPython.display import Image, clear_output
import torch
from shutil import copyfile



from collections import Counter

from math import *

from scipy.linalg import norm
import scipy.ndimage

from ipywidgets.widgets import *
import ipywidgets as widgets

import plotly
import plotly.express as px
from plotly.graph_objs import *

#import chart_studio.plotly as py

from mpl_toolkits.mplot3d import Axes3D

import cv2


from skimage import io, img_as_float, img_as_ubyte, color, exposure, measure, morphology, data, transform
from skimage.measure import label, regionprops
from skimage.color import rgb2lab, deltaE_cie76
from skimage.util import compare_images 

from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

import pydicom 
from pydicom.data import get_testdata_file
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom import dcmread
import gdcm

%matplotlib inline

import os
    #for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))


## 2. Collect data

#### Read Files

In [ ]:
train_image_level = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')
train_study_level = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv')

In [ ]:
print( "'train_image_level's data has ",train_image_level.shape[0], 'samples')
print( "& 'train_study_level's data has ",train_study_level.shape[0], 'samples')

In [ ]:
train_image_level.head(3)

In [ ]:
train_study_level.head(3)

#### Merge study csv to image csv 

In [ ]:
#Merge study csv to image csv.
train_study_level['StudyInstanceUID'] = train_study_level['id'].apply(lambda x: x.replace('_study', ''))
train_image_level = train_image_level.merge(train_study_level, on='StudyInstanceUID')
complete_data = train_image_level.drop(['id_x','id_y'], axis=1)
complete_data.head(3)

#### Merge image path to the train_image_level dataframe

In [ ]:
# Merge image path to the train_image_level dataframe.
train_dir = '/kaggle/input/siim-covid19-detection/train'
test_dir = '/kaggle/input/siim-covid19-detection/test'

training_paths = []

for sid in tqdm(complete_data['StudyInstanceUID']):
    training_paths.append(glob.glob(os.path.join(train_dir,sid+"/*/*"))[0])

complete_data['path'] = training_paths

complete_data.head(3)

In [ ]:
# Read full text in table
pd.set_option('display.max_colwidth', -1)
complete_data.head(3)

#### Read image file (.dcm)

In [ ]:
#training_paths = []
ds = pydicom.dcmread(training_paths[0])
ds


#### Add some DCM's conlumns to "Complete Data"

In [ ]:
def process_dicom(dicom_obj):
    pixel_data=(0x7fe0, 0x0010) #ignore the pixel data
    data_dict={}
    for x in dicom_obj:
        if x.tag==pixel_data:
            continue
        value=dicom_obj[x.tag].value
        name=x.name
        data_dict[name]=value
    return data_dict

In [ ]:
columns_reordered = list(complete_data.columns)
complete_data = complete_data[columns_reordered]

dicom_dict={}
needed_columns=["Patient ID","Patient's Sex","Body Part Examined","Imager Pixel Spacing",
                "Photometric Interpretation","Modality"]
# "Study Instance UID","Study ID"

for i,x in tqdm(complete_data.iterrows()):
    dicom_obj=pydicom.dcmread(x['path'],stop_before_pixels=True)
    dicom_obj_dict=process_dicom(dicom_obj)
    for key in dicom_obj_dict:
        if type(dicom_obj_dict[key])==list:
            continue
        if key in needed_columns:
            if key not in dicom_dict:
                dicom_dict[key]=[]
            dicom_dict[key].append(dicom_obj_dict[key])
            
for col in needed_columns:
    complete_data[col]=dicom_dict[col]


## 3. Review

#### Review complete file

In [ ]:
complete_data.head(1)

#### Review Patient's Sex

In [ ]:
Negative = complete_data[complete_data['Negative for Pneumonia']==1]
Typical = complete_data[complete_data['Typical Appearance']==1]
Indeterminate = complete_data[complete_data['Indeterminate Appearance']==1]
Atypical = complete_data[complete_data['Atypical Appearance']==1]

data = pd.DataFrame({'Typical Appearance': Typical["Patient's Sex"].value_counts(),
              'Negative for Pneumonia': Negative["Patient's Sex"].value_counts(),
              'Indeterminate Appearance': Indeterminate["Patient's Sex"].value_counts(),
              'Atypical Appearance': Atypical["Patient's Sex"].value_counts()
             })
data_t = data.transpose()
ax = data_t.plot.bar(stacked = True, width=0.8,figsize=(15,5), title="Annotation Per Class = 1")

plt.grid(axis='y')
plt.xticks(rotation=0)
plt.show()

#### Review Body Part Examined
##### I need to review some image of each "Body Part Examined"

In [ ]:
complete_data['Body Part Examined'].value_counts()

##### Check


###### after check each body part examined, ABDOMEN image can not show.
###### there are some error.
###### so I delete ABDOMEN's data from complete_data.

In [ ]:
#complete_data = complete_data[complete_data['Body Part Examined']!='ABDOMEN']
#complete_data['Body Part Examined'].value_counts()

#### Review Image

In [ ]:
label_appearance = list(complete_data.columns)[3:7]
row_img = 2
col_img = len(label_appearance)

c= 0
color = 'gray'
ds_pos = [0 for p in range(row_img)]

fig,ax = plt.subplots(row_img,col_img, figsize=(25,row_img*4))

for i in label_appearance:
    for r in range(row_img):
        link = complete_data[complete_data[i] == 1]['path'].iloc[r]
        ds_pos[r] = pydicom.dcmread(link).pixel_array
        ax[r,c].imshow(ds_pos[r], cmap=color)
        
      
    ax[0,c].set_title(i, font='Serif', fontsize=20)
    c +=1

## 4. Data Exploration (lebel group)

#### Photometric Interpretation have 2 types.
        * MONOCHROME1 : indicates that the greyscale ranges from bright to dark with ascending pixel values
        * MONOCHROME2 : ranges from dark to bright with ascending pixel values.
        
#### Modality have 2 types. 
        * CR
        * DX
    
   refer: **Digital Radiography versus Computed Radiography**
   
   https://www.news-medical.net/health/Digital-Radiography-versus-Computed-Radiography.aspx

#### I will review by seperate each label appearances with Photometric Interpretation and Modality

### Negative for Pneumonia group 

In [ ]:
Negative_Pneumonia = complete_data[complete_data['Negative for Pneumonia']==1]
Negative_Pneumonia1_cr = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME1') &
                                            (Negative_Pneumonia['Modality'] == 'CR')].reset_index()
Negative_Pneumonia1_dx = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME1') &
                                            (Negative_Pneumonia['Modality'] == 'DX')].reset_index()
Negative_Pneumonia2_cr = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME2') &
                                            (Negative_Pneumonia['Modality'] == 'CR')].reset_index()
Negative_Pneumonia2_dx = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME2') &
                                            (Negative_Pneumonia['Modality'] == 'DX')].reset_index()

Nagative = [Negative_Pneumonia1_cr, Negative_Pneumonia1_dx, Negative_Pneumonia2_cr, Negative_Pneumonia2_dx]

In [ ]:
row = 2
col = len(Nagative)
pos = 0
i = 0
#fig, ax = plt.subplots(row,col, figsize=(25,row*5))

for data in Nagative:
    for pos in range(row): 
        link = data['path'].iloc[pos]
        ds_pos = pydicom.dcmread(link).pixel_array
        #ax[pos,i].imshow(ds_pos,cmap='gray')
    i += 1
    
  #  ax[0,0].set_title('Negative with MONOCHROME1 & CR', font='Serif', fontsize=15)
  #  ax[0,1].set_title('Negative with MONOCHROME1 & DX', font='Serif', fontsize=15)
  #  ax[0,2].set_title('Negative with MONOCHROME2 & CR', font='Serif', fontsize=15)
  #  ax[0,3].set_title('Negative with MONOCHROME2 & DX', font='Serif', fontsize=15)
        


### Typical group 


In [ ]:
Typical_Appearance = complete_data[complete_data['Typical Appearance'] == 1].reset_index()
Typical_Appearance1_cr = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME1') &
                                            (Typical_Appearance['Modality'] == 'CR')].reset_index()
Typical_Appearance1_dx = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME1') &
                                            (Typical_Appearance['Modality'] == 'DX')].reset_index()
Typical_Appearance2_cr = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME2') &
                                            (Typical_Appearance['Modality'] == 'CR')].reset_index()
Typical_Appearance2_dx = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME2') &
                                            (Typical_Appearance['Modality'] == 'DX')].reset_index()

typical = [Typical_Appearance1_cr,Typical_Appearance1_dx,Typical_Appearance2_cr,Typical_Appearance2_dx]

In [ ]:
row = 2
col = len(typical)
pos = 0
i = 0
#fig, ax = plt.subplots(row,col, figsize=(25,row*5))

for data in typical:
    for pos in range(row): 
        
        link = data['path'].iloc[pos]
        ds_pos = [0 for p in range(pos+1)]
        ds_pos = pydicom.dcmread(link).pixel_array

        
        #ax[pos,i].imshow(ds_pos,cmap='gray')
    i += 1
    
 #   ax[0,0].set_title('Typical with MONOCHROME1 & CR', font='Serif', fontsize=15)
 #   ax[0,1].set_title('Typical with MONOCHROME1 & DX', font='Serif', fontsize=15)
 #   ax[0,2].set_title('Typical with MONOCHROME2 & CR', font='Serif', fontsize=15)
 #   ax[0,3].set_title('Typical with MONOCHROME2 & DX', font='Serif', fontsize=15)
        


### Indeterminate group

In [ ]:
Indeterminate_Appearance = complete_data[complete_data['Indeterminate Appearance'] == 1].reset_index()
Indeterminate_Appearance1_cr = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME1') &
                                                        (Indeterminate_Appearance['Modality'] == 'CR')].reset_index()
Indeterminate_Appearance1_dx = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME1') &
                                                        (Indeterminate_Appearance['Modality'] == 'DX')].reset_index()
Indeterminate_Appearance2_cr = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME2') &
                                                        (Indeterminate_Appearance['Modality'] == 'CR')].reset_index()
Indeterminate_Appearance2_dx = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME2') &
                                                        (Indeterminate_Appearance['Modality'] == 'DX')].reset_index()
indeterminate = [Indeterminate_Appearance1_cr,Indeterminate_Appearance1_dx,Indeterminate_Appearance2_cr,Indeterminate_Appearance2_dx]

In [ ]:
row = 2
col = len(indeterminate)
pos = 0
i = 0
#fig, ax = plt.subplots(row,col, figsize=(25,row*5))

for data in indeterminate:
    for pos in range(row): 
        
        link = data['path'].iloc[pos]
        ds_pos = [0 for p in range(pos+1)]
        ds_pos = pydicom.dcmread(link).pixel_array

        
       # ax[pos,i].imshow(ds_pos,cmap='gray')
    i += 1
    
    #ax[0,0].set_title('Indeterminate with MONOCHROME1 & CR', font='Serif', fontsize=15)
    #ax[0,1].set_title('Indeterminate with MONOCHROME1 & DX', font='Serif', fontsize=15)
    #ax[0,2].set_title('Indeterminate with MONOCHROME2 & CR', font='Serif', fontsize=15)
    #ax[0,3].set_title('Indeterminate with MONOCHROME2 & DX', font='Serif', fontsize=15)

### Atypical group

In [ ]:
Atypical_Appearance = complete_data[complete_data['Atypical Appearance'] == 1].reset_index()
Atypical_Appearance1_cr = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME1')&
                                             (Atypical_Appearance['Modality'] == 'CR')].reset_index()
Atypical_Appearance1_dx = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME1')&
                                             (Atypical_Appearance['Modality'] == 'DX')].reset_index()
Atypical_Appearance2_cr = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME2')&
                                             (Atypical_Appearance['Modality'] == 'CR')].reset_index()
Atypical_Appearance2_dx = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME2')&
                                             (Atypical_Appearance['Modality'] == 'DX')].reset_index()
atypical = [Atypical_Appearance1_cr,Atypical_Appearance1_dx,Atypical_Appearance2_cr,Atypical_Appearance2_dx]

In [ ]:
row = 2
col = len(atypical)
pos = 0
i = 0
#fig, ax = plt.subplots(row,col, figsize=(25,row*5))

for data in atypical:
    for pos in range(row): 
        
        link = data['path'].iloc[pos]
        ds_pos = [0 for p in range(pos+1)]
        ds_pos = pydicom.dcmread(link).pixel_array

        
       # ax[pos,i].imshow(ds_pos,cmap='gray')
    i += 1
    
  #  ax[0,0].set_title('Atypical with MONOCHROME1 & CR', font='Serif', fontsize=15)
  #  ax[0,1].set_title('Atypical with MONOCHROME1 & DX', font='Serif', fontsize=15)
  #  ax[0,2].set_title('Atypical with MONOCHROME2 & CR', font='Serif', fontsize=15)
  #  ax[0,3].set_title('Atypical with MONOCHROME2 & DX', font='Serif', fontsize=15)

## 5. Data Exploation (Photometric Interpretation and Modality types.)

### MONOCROME1 & CR

In [ ]:
order = 3
color = 'gray'
fig, ax = plt.subplots(order,4, figsize=(30,order*7))


####################################################
#MONOCROME1 & CR
Negative_Pneumonia1_cr = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME1') &
                                            (Negative_Pneumonia['Modality'] == 'CR')].reset_index()


for pos in range(order):
    
 
    link = Negative_Pneumonia1_cr['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    ax[pos,0].imshow(ds_pos[pos],cmap=color)      
    ax[0,0].set_title('Negative', font='Serif', fontsize=20)


####################################################
#MONOCROME1 & CR
Typical_Appearance1_cr = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME1') &
                                            (Typical_Appearance['Modality'] == 'CR')].reset_index()


for pos in range(order):
    
    boxes = ast.literal_eval(Typical_Appearance1_cr.loc[pos,'boxes'])
    link = Typical_Appearance1_cr['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='g', fc='none', lw=3)
        ax[pos,1].add_patch(p)
    ax[pos,1].imshow(ds_pos[pos],cmap=color)      
    ax[0,1].set_title('Typical', font='Serif', fontsize=20)

####################################################
#MONOCROME1 & CR
Indeterminate_Appearance1_cr = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME1') &
                                                        (Indeterminate_Appearance['Modality'] == 'CR')].reset_index()

for pos in range(order):
    
    boxes = ast.literal_eval(Indeterminate_Appearance1_cr.loc[pos,'boxes'])
    link = Indeterminate_Appearance1_cr['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='b', fc='none', lw=3)
        ax[pos,2].add_patch(p)
    ax[pos,2].imshow(ds_pos[pos],cmap=color)      
    ax[0,2].set_title('Indeterminate', font='Serif', fontsize=20)
    
    
####################################################
#MONOCROME1 & CR
Atypical_Appearance1_cr = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME1')&
                                             (Atypical_Appearance['Modality'] == 'CR')].reset_index()

for pos in range(order):
    
    boxes = ast.literal_eval(Atypical_Appearance1_cr.loc[pos,'boxes'])
    link = Atypical_Appearance1_cr['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='r', fc='none', lw=3)
        ax[pos,3].add_patch(p)
    ax[pos,3].imshow(ds_pos[pos],cmap=color)      
    ax[0,3].set_title('Atypical', font='Serif', fontsize=20)


### MONOCROME1 & DX

In [ ]:
order = 2
color = 'gray'
fig, ax = plt.subplots(order,4, figsize=(30,order*7))


####################################################
#MONOCROME1 & DX
Negative_Pneumonia1_dx = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME1') &
                                            (Negative_Pneumonia['Modality'] == 'DX')].reset_index()


for pos in range(order):
    
 
    link = Negative_Pneumonia1_dx['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    ax[pos,0].imshow(img,cmap=color)      
    ax[0,0].set_title('Negative', font='Serif', fontsize=20)


####################################################
#MONOCROME1 & DX
Typical_Appearance1_dx = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME1') &
                                            (Typical_Appearance['Modality'] == 'DX')].reset_index()


for pos in range(order):
    
    boxes = ast.literal_eval(Typical_Appearance1_dx.loc[pos,'boxes'])
    link = Typical_Appearance1_dx['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='g', fc='none', lw=3)
        ax[pos,1].add_patch(p)
    ax[pos,1].imshow(img,cmap=color)      
    ax[0,1].set_title('Typical', font='Serif', fontsize=20)

####################################################
#MONOCROME1 & DX
Indeterminate_Appearance1_dx = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME1') &
                                                        (Indeterminate_Appearance['Modality'] == 'DX')].reset_index()

for pos in range(order):
    
    boxes = ast.literal_eval(Indeterminate_Appearance1_dx.loc[pos,'boxes'])
    link = Indeterminate_Appearance1_dx['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='b', fc='none', lw=3)
        ax[pos,2].add_patch(p)
    ax[pos,2].imshow(img,cmap=color)      
    ax[0,2].set_title('Indeterminate', font='Serif', fontsize=20)
    
    
####################################################
#MONOCROME1 & DX
Atypical_Appearance1_dx = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME1')&
                                             (Atypical_Appearance['Modality'] == 'DX')].reset_index()

for pos in range(order):
    
    boxes = ast.literal_eval(Atypical_Appearance1_dx.loc[pos,'boxes'])
    link = Atypical_Appearance1_dx['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='r', fc='none', lw=3)
        ax[pos,3].add_patch(p)
    ax[pos,3].imshow(img,cmap=color)      
    ax[0,3].set_title('Atypical', font='Serif', fontsize=20)


### MONOCROME2 & CR

In [ ]:
order = 2
color = 'gray'
fig, ax = plt.subplots(order,4, figsize=(30,order*7))


####################################################
#MONOCROME2 & CR
Negative_Pneumonia2_cr = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME2') &
                                            (Negative_Pneumonia['Modality'] == 'CR')].reset_index()


for pos in range(order):
    
 
    link = Negative_Pneumonia2_cr['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    ax[pos,0].imshow(img,cmap=color)      
    ax[0,0].set_title('Negative', font='Serif', fontsize=20)


####################################################
#MONOCROME2 & CR
Typical_Appearance2_cr = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME2') &
                                            (Typical_Appearance['Modality'] == 'CR')].reset_index()


for pos in range(order):
    
    boxes = ast.literal_eval(Typical_Appearance2_cr.loc[pos,'boxes'])
    link = Typical_Appearance2_cr['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='g', fc='none', lw=3)
        ax[pos,1].add_patch(p)
    ax[pos,1].imshow(img,cmap=color)      
    ax[0,1].set_title('Typical', font='Serif', fontsize=20)

####################################################
#MONOCROME2 & CR
Indeterminate_Appearance2_cr = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME2') &
                                                        (Indeterminate_Appearance['Modality'] == 'CR')].reset_index()

for pos in range(order):
    
    boxes = ast.literal_eval(Indeterminate_Appearance2_cr.loc[pos,'boxes'])
    link = Indeterminate_Appearance2_cr['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='b', fc='none', lw=3)
        ax[pos,2].add_patch(p)
    ax[pos,2].imshow(img,cmap=color)      
    ax[0,2].set_title('Indeterminate', font='Serif', fontsize=20)
    
    
####################################################
#MONOCROME2 & CR
Atypical_Appearance2_cr = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME2')&
                                             (Atypical_Appearance['Modality'] == 'CR')].reset_index()

for pos in range(order):
    
    boxes = ast.literal_eval(Atypical_Appearance2_cr.loc[pos,'boxes'])
    link = Atypical_Appearance2_cr['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='r', fc='none', lw=3)
        ax[pos,3].add_patch(p)
    ax[pos,3].imshow(img,cmap=color)      
    ax[0,3].set_title('Atypical', font='Serif', fontsize=20)


### MONOCROME2 & DX

In [ ]:
order = 2
color = 'gray'
fig, ax = plt.subplots(order,4, figsize=(30,order*7))


####################################################
#MONOCROME2 & DX
Negative_Pneumonia2_dx = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME2') &
                                            (Negative_Pneumonia['Modality'] == 'DX')].reset_index()


for pos in range(order):
    
 
    link = Negative_Pneumonia2_dx['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    ax[pos,0].imshow(img,cmap=color)      
    ax[0,0].set_title('Negative', font='Serif', fontsize=20)


####################################################
#MONOCROME2 & DX
Typical_Appearance2_dx = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME2') &
                                            (Typical_Appearance['Modality'] == 'CR')].reset_index()


for pos in range(order):
    
    boxes = ast.literal_eval(Typical_Appearance2_dx.loc[pos,'boxes'])
    link = Typical_Appearance2_dx['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='g', fc='none', lw=3)
        ax[pos,1].add_patch(p)
    ax[pos,1].imshow(img,cmap=color)      
    ax[0,1].set_title('Typical', font='Serif', fontsize=20)

####################################################
#MONOCROME2 & DX
Indeterminate_Appearance2_dx = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME2') &
                                                        (Indeterminate_Appearance['Modality'] == 'CR')].reset_index()

for pos in range(order):
    
    boxes = ast.literal_eval(Indeterminate_Appearance2_dx.loc[pos,'boxes'])
    link = Indeterminate_Appearance2_dx['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='b', fc='none', lw=3)
        ax[pos,2].add_patch(p)
    ax[pos,2].imshow(img,cmap=color)      
    ax[0,2].set_title('Indeterminate', font='Serif', fontsize=20)
    
    
####################################################
#MONOCROME2 & DX
Atypical_Appearance2_dx = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME2')&
                                             (Atypical_Appearance['Modality'] == 'CR')].reset_index()

for pos in range(order):
    
    boxes = ast.literal_eval(Atypical_Appearance2_dx.loc[pos,'boxes'])
    link = Atypical_Appearance2_dx['path'].iloc[pos]
    ds_pos = [0 for p in range(pos+1)]
    ds_pos[pos] = pydicom.dcmread(link).pixel_array
    img = exposure.equalize_hist(ds_pos[pos])

    for box in boxes:
        p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='r', fc='none', lw=3)
        ax[pos,3].add_patch(p)
    ax[pos,3].imshow(img,cmap=color)      
    ax[0,3].set_title('Atypical', font='Serif', fontsize=20)


## 6. MONOCROME1

#### Gray will be in the range 40 - 200.
##### I will divide the levels in 10 steps.
##### Firstly, I will one image of each types. 


In [ ]:
#### change image type from uint16 to uint8 by read_xray

def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
        #data = exposure.equalize_adapthist(data)
    else:
        data = dicom.pixel_array
        #data = exposure.equalize_adapthist(data)
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def grayRange(lower, upper,img ,hsv, color, patient_id, data):
    #cnt = count 
    cnt_L = 0
    cnt_R = 0
    mask = cv2.inRange(hsv, lower, upper)
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #cv2.drawContours(img,contours, -1, color, 3)
    
    # fig size = 1200 x 1000
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)    
        if 0 < x < 600 and 100 < y < 800 and 0 < w < 1000 and 0 < h < 1000 :
            area = w*h
            if 0 < area < 10000:
                #cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
                cv2.drawContours(img,contour, -2, color, 3)
                cnt_L += 1
        
        elif 600 < x < 1100 and 100 < y < 800 and 0 < w < 1000 and 0 < h < 1000 :
            area = w*h
            if 0 < area < 10000:
                #cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
                cv2.drawContours(img,contour, -2, color, 3)
                cnt_R += 1               
    
    count_L.append(cnt_L)
    count_R.append(cnt_R)
        
    
            
   
    return data, count_L, count_R
    
   
    

In [ ]:
def chkGray1(path_img, hsv, img, re_img_gray, patient_id, data, count_L, count_R):
    count = 0
    
    
    lower1 = np.array([0,0,0])
    upper1 = np.array([0,0,20])
    color1 = (0,0,255)

    lower2 = np.array([0,0,20])
    upper2 = np.array([0,0,30])
    color2 = (0,0,255)

    lower3 = np.array([0,0,30])
    upper3 = np.array([0,0,40])
    color3 = (0,0,255)

    lower4 = np.array([0,0,40])
    upper4 = np.array([0,0,50])
    color4 = (0,255,0)

    lower5 = np.array([0,0,50])
    upper5 = np.array([0,0,60])
    color5 = (0,255,0)

    lower6 = np.array([0,0,60])
    upper6 = np.array([0,0,70])
    color6 = (0,255,0)

    lower7 = np.array([0,0,70])
    upper7 = np.array([0,0,80])
    color7 = (0,255,0)

    lower8 = np.array([0,0,80])
    upper8 = np.array([0,0,90])
    color8 = (0,255,0)

    lower9 = np.array([0,0,90])
    upper9 = np.array([0,0,100])
    color9 = (255,255,187)

    lower10 = np.array([0,0,100])
    upper10 = np.array([0,0,110])
    color10 = (255,255,187)

    lower11 = np.array([0,0,110])
    upper11 = np.array([0,0,120])
    color11 = (255,255,0)

    lower12 = np.array([0,0,120])
    upper12 = np.array([0,0,130])
    color12 = (255,255,0)

    lower13 = np.array([0,0,130])
    upper13 = np.array([0,0,140])
    color13 = (255,128,0)

    lower14 = np.array([0,0,140])
    upper14 = np.array([0,0,150])
    color14 = (255,128,0)

    lower15 = np.array([0,0,150])
    upper15 = np.array([0,0,160])
    color15 = (255,128,0)

    lower16 = np.array([0,0,160])
    upper16 = np.array([0,0,170])
    color16 = (255,0,255)

    lower17 = np.array([0,0,170])
    upper17 = np.array([0,0,180])
    color17 = (255,0,255)
    
    lower18 = np.array([0,0,180])
    upper18 = np.array([0,0,190])
    color18 = (0,0,0)
        
    lower19 = np.array([0,0,190])
    upper19 = np.array([0,0,200])
    color19 = (255,0,0)
    
    lower20 = np.array([0,0,200])
    upper20 = np.array([0,0,210])
    color20 = (255,0,0)
    
    lower21 = np.array([0,0,210])
    upper21 = np.array([0,0,220])
    color21 = (136,0,0)
    
    lower22 = np.array([0,0,220])
    upper22 = np.array([0,0,230])
    color22 = (136,0,0)
    
    lower23 = np.array([0,0,230])
    upper23 = np.array([0,0,240])
    color23 = (136,0,0)
    
    lower24 = np.array([0,0,240])
    upper24 = np.array([0,0,255])
    color24 = (136,0,0)
    

    grayRange(lower1, upper1, img, hsv, color1, patient_id, data = 'g1')   #น้ำเงิน
    data.append('0')
    grayRange(lower2, upper2, img, hsv, color2, patient_id, data = 'g2')  #น้ำเงิน
    data.append('1')
    grayRange(lower3, upper3, img, hsv, color3, patient_id, data = 'g3')   #น้ำเงิน
    data.append('2')
    grayRange(lower4, upper4, img, hsv, color4, patient_id, data = 'g4')   #เขียว
    data.append('3')
    grayRange(lower5, upper5, img, hsv, color5, patient_id, data = 'g5')   #เขียว
    data.append('4')
    grayRange(lower6, upper6, img, hsv, color6, patient_id, data = 'g6')   #เขียว
    data.append('5')
    grayRange(lower7, upper7, img, hsv, color7, patient_id, data = 'g7')   #เขียว
    data.append('6')
    grayRange(lower8, upper8, img, hsv, color8, patient_id, data = 'g8')   #เขียว
    data.append('7')
    grayRange(lower9, upper9, img, hsv, color9, patient_id, data = 'g9')   #เหลืองอ่อน
    data.append('8')
    grayRange(lower10, upper10, img, hsv, color10, patient_id, data = 'g10' )  #เหลืองอ่อน
    data.append('9')
    grayRange(lower11, upper11, img, hsv, color11, patient_id, data = 'g11')  #เหลือง
    data.append('10')
    grayRange(lower12, upper12, img, hsv, color12, patient_id, data = 'g12')  #เหลือง
    data.append('11')
    grayRange(lower13, upper13, img, hsv, color13, patient_id, data = 'g13')  #ส้ม
    data.append('12')
    grayRange(lower14, upper14, img, hsv, color14, patient_id, data = 'g14')  #้ส้ม
    data.append('13')
    grayRange(lower15, upper15, img, hsv, color15, patient_id, data = 'g15')  #ส้ม
    data.append('14')
    grayRange(lower16, upper16, img, hsv, color16, patient_id, data = 'g16')  #ชมพู
    data.append('15')
    grayRange(lower17, upper17, img, hsv, color17, patient_id, data = 'g17')  #ชมพู
    data.append('16')
    grayRange(lower18, upper18, img, hsv, color18, patient_id, data = 'g18')  #แดง
    data.append('17')
    grayRange(lower19, upper19, img, hsv, color19, patient_id, data = 'g19')  #แดง
    data.append('18')
    grayRange(lower20, upper20, img, hsv, color20, patient_id, data = 'g20')  #แดง
    data.append('19')
    #grayRange(lower21, upper21, img, hsv, color21, patient_id, data = 'g21')  #เลือดหมู
    #grayRange(lower22, upper22, img, hsv, color22, patient_id, data = 'g22')  #เลือดหมู
    #grayRange(lower23, upper23, img, hsv, color23, patient_id, data = 'g23')  #เลือดหมู
    #grayRange(lower24, upper24, img, hsv, color24, patient_id, data = 'g24')  #เลือดหมู
    
    
    return re_img_gray, data, patient_id, count_L, count_R

In [ ]:

Negative_Pneumonia1 = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME1')].reset_index()
Typical_Appearance1 = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME1')].reset_index()
Indeterminate_Appearance1 = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME1')].reset_index()
Atypical_Appearance1 = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME1')].reset_index()

print('Negative_Pneumonia1: ',Negative_Pneumonia1['path'].shape[0], 'images')
print('Typical_Appearance1: ',Typical_Appearance1['path'].shape[0], 'images')
print('Indeterminate_Appearance1: ',Indeterminate_Appearance1['path'].shape[0], 'images')
print('Atypical_Appearance1: ',Atypical_Appearance1['path'].shape[0], 'images')

In [ ]:
#target => complete_data                            

#MONOCROME1&CR => Negative_Pneumonia1_cr, Typical_Appearance1_cr ,Indeterminate_Appearance1_cr, Atypical_Appearance1_cr

Negative_Pneumonia1 = Negative_Pneumonia[(Negative_Pneumonia['Photometric Interpretation']=='MONOCHROME1')].reset_index()
Typical_Appearance1 = Typical_Appearance[(Typical_Appearance['Photometric Interpretation']=='MONOCHROME1')].reset_index()
Indeterminate_Appearance1 = Indeterminate_Appearance[(Indeterminate_Appearance['Photometric Interpretation']=='MONOCHROME1')].reset_index()
Atypical_Appearance1 = Atypical_Appearance[(Atypical_Appearance['Photometric Interpretation']=='MONOCHROME1')].reset_index()

patient_id = []
data = []
count_L = []
count_R = []
x_position = []
status = []
sex =[]

test1 = Negative_Pneumonia1
for i in range(Negative_Pneumonia1['path'].shape[0]):   
    check = test1.iloc[i]
    if check['Photometric Interpretation']=='MONOCHROME1':
        img_path = check['path']  
        imgC = read_xray(img_path)   #change from 16bit color to 8bit color
        img_gray = cv2.cvtColor(imgC, cv2.COLOR_BGR2RGB)   # make Gray color
        re_img = cv2.resize(img_gray,(1200,1000))
        hsv = cv2.cvtColor(re_img, cv2.COLOR_BGR2HSV)
        chkGray1(img, hsv, re_img, re_img, check['Patient ID'], data, count_L, count_R)
        for id in range(20):
            patient_id.append(check['Patient ID'])
            sex.append(check["Patient's Sex"])
            #status.append('neg')
            status.append(0)

        #print(patient_id)
        #print(status)
        #print(data)
        #print(count_L)
        #print(count_R)
    #plt.imshow(re_img) 

test2 = Typical_Appearance1
for i in range(Typical_Appearance1['path'].shape[0]):   
    check = test2.iloc[i]
    if check['Photometric Interpretation']=='MONOCHROME1':
        img_path = check['path']  
        imgC = read_xray(img_path)   #change from 16bit color to 8bit color
        img_gray = cv2.cvtColor(imgC, cv2.COLOR_BGR2RGB)   # make Gray color
        re_img = cv2.resize(img_gray,(1200,1000))
        hsv = cv2.cvtColor(re_img, cv2.COLOR_BGR2HSV)
        chkGray1(img, hsv, re_img, re_img, check['Patient ID'], data, count_L, count_R)
        for id in range(20):
            patient_id.append(check['Patient ID'])
            sex.append(check["Patient's Sex"])
            #status.append('typ')
            status.append(1)
    
    

test3 = Indeterminate_Appearance1
for i in range(Indeterminate_Appearance1['path'].shape[0]):   
    check = test3.iloc[i]
    if check['Photometric Interpretation']=='MONOCHROME1':
        img_path = check['path']  
        imgC = read_xray(img_path)   #change from 16bit color to 8bit color
        img_gray = cv2.cvtColor(imgC, cv2.COLOR_BGR2RGB)   # make Gray color
        re_img = cv2.resize(img_gray,(1200,1000))
        hsv = cv2.cvtColor(re_img, cv2.COLOR_BGR2HSV)
        chkGray1(img, hsv, re_img, re_img, check['Patient ID'], data, count_L, count_R)
        for id in range(20):
            patient_id.append(check['Patient ID'])
            sex.append(check["Patient's Sex"])
            #status.append('In')
            status.append(2)
            

test4 = Atypical_Appearance1
for i in range(Atypical_Appearance1['path'].shape[0]):   
    check = test4.iloc[i]
    if check['Photometric Interpretation']=='MONOCHROME1':
        img_path = check['path']  
        imgC = read_xray(img_path)   #change from 16bit color to 8bit color
        img_gray = cv2.cvtColor(imgC, cv2.COLOR_BGR2RGB)   # make Gray color
        re_img = cv2.resize(img_gray,(1200,1000))
        hsv = cv2.cvtColor(re_img, cv2.COLOR_BGR2HSV)
        chkGray1(img, hsv, re_img, re_img, check['Patient ID'], data, count_L, count_R)
        for id in range(20):
            patient_id.append(check['Patient ID'])
            sex.append(check["Patient's Sex"])
            #status.append('Atyp')
            status.append(3)
    
    
    
df_check = pd.DataFrame({'patient_id': patient_id,
                         'sex': sex,
                         'status': status,
                         'ref': data,
                         'Left': count_L,
                         'Right': count_R                 
                        })

#df_check
#b,b,b,g,g,g,g,g,yl,yl,y,y,o,o,o,p,p,r,r
df_check.shape

In [ ]:
def des(des):
    if des == 0:
        return 'neg'
    elif des == 1:
        return 'typ'
    elif des == 2:
        return 'In'
    elif des == 3:
        return 'Atyp'
    else:
        return 'none'
    
def sex(sex):
    if sex == 'M':
        return 0
    elif sex == 'F':
        return 1
    else:
        return 'none'

df_check['description']=df_check['status'].apply(des)
df_check['sex_value']=df_check['sex'].apply(sex)
df_check.head()

In [ ]:

#replace Nan and Infiniy value     
#df_check['ratio'] = df_check['ratio'].fillna(0)   #replace Nan
#df_check['ratio'] = df_check['ratio'].replace([np.inf, -np.inf], 99) #replace infinity

df_check.loc[df_check.Left==0, 'Left'] = 1
df_check.loc[df_check.Right==0, 'Right'] = 1
df_check['ratio'] = df_check['Left']/df_check['Right']

df_check.head()

In [ ]:
col = 4   #fix
row = 10

gs = GridSpec(row,col)
fig = plt.figure(figsize=(30,5*row))
count_L = []
count_R = []

for i in range(row):
        
    path_img_neg = Negative_Pneumonia1['path'].iloc[(i)]
    patient_id_n = Negative_Pneumonia1['Patient ID'].iloc[i]

    #change from 16bit color to 8bit color
    imageC_n = read_xray(path_img_neg)
    
    # make Gray color
    img_gray = cv2.cvtColor(imageC_n, cv2.COLOR_BGR2RGB)
        
    #reduce shape (1200,1000)
    re_img_gray_n = cv2.resize(img_gray,(1200,1000))
    img_n = re_img_gray_n
       
    hsv_n = cv2.cvtColor(img_n, cv2.COLOR_BGR2HSV)
    chkGray1(path_img_neg, hsv_n, img_n, re_img_gray_n, patient_id_n, data, count_L, count_R)
        
    fig.add_subplot(gs[i,0]).set_title('Nagative')
    fig.add_subplot(gs[i,0]).imshow(re_img_gray_n)

for i in range(row):
    path_img_typ= Typical_Appearance1['path'].iloc[i]
    patient_id_t = Typical_Appearance1['Patient ID'].iloc[i]
    
    #change from 16bit color to 8bit color
    imageC_typ = read_xray(path_img_typ)
    
    # make Gray color
    img_gray = cv2.cvtColor(imageC_typ, cv2.COLOR_BGR2RGB)
    
    #reduce shape (1200,1000)
    re_img_gray_typ = cv2.resize(img_gray,(1200,1000))
    img_typ = re_img_gray_typ
    hsv_t = cv2.cvtColor(img_typ, cv2.COLOR_BGR2HSV)

    chkGray1(path_img_typ, hsv_t, img_typ, re_img_gray_typ, patient_id_t, data, count_L, count_R)
    
    fig.add_subplot(gs[i,1]).set_title('Typical')
    fig.add_subplot(gs[i,1]).imshow(img_typ)

for i in range(row):
 
    path_img_in = Indeterminate_Appearance1['path'].iloc[i]
    patient_id_in = Indeterminate_Appearance1['Patient ID'].iloc[i]

    #change from 16bit color to 8bit color
    imageC_in = read_xray(path_img_in)
    # make Gray color
    img_gray = cv2.cvtColor(imageC_in, cv2.COLOR_BGR2RGB)
    #reduce shape (1200,1000)
    re_img_gray_in = cv2.resize(img_gray,(1200,1000))
    img_in = re_img_gray_in
    hsv_in = cv2.cvtColor(img_in, cv2.COLOR_BGR2HSV)

    chkGray1(path_img_in, hsv_in, img_in, re_img_gray_in, patient_id_in, data, count_L, count_R)
    
    fig.add_subplot(gs[i,2]).set_title('Indeterminate')
    fig.add_subplot(gs[i,2]).imshow(img_in)

for i in range(row):
    path_img_a = Atypical_Appearance1['path'].iloc[i]
    patient_id_a = Atypical_Appearance1['Patient ID'].iloc[i]

    #change from 16bit color to 8bit color
    imageC_a = read_xray(path_img_a)
    # make Gray color
    img_gray = cv2.cvtColor(imageC_a, cv2.COLOR_BGR2RGB)
    #reduce shape (1200,1000)
    re_img_gray_a = cv2.resize(img_gray,(1200,1000))
    img_a = re_img_gray_a
    hsv_a = cv2.cvtColor(img_a, cv2.COLOR_BGR2HSV)
    
    chkGray1(path_img_a, hsv_a, img_a, re_img_gray_a, patient_id_a, data, count_L, count_R)

    fig.add_subplot(gs[i,3]).set_title('Atypical')
    fig.add_subplot(gs[i,3]).imshow(img_a)


In [ ]:


sns.catplot(data=df_check, x='ref', y='Left', hue='description')
sns.lineplot(data=df_check, x='ref', y='Left', hue='description')
sns.catplot(data=df_check, x='ref', y='Right', hue='description')
sns.lineplot(data=df_check, x='ref', y='Right', hue='description')
#sns.catplot(data=df_chk2, x='ref', y='ratio', hue='description')
#sns.catplot(data=df_chk3, x='ref', y='ratio', hue='description')

plt.show()

In [ ]:
g = sns.PairGrid(df_check, vars=['ref', 'Left', 'Right','ratio'],
                 hue='description', palette='RdBu_r')
g.map(plt.scatter, alpha=0.8)
g.add_legend();

In [ ]:
#fig = px.scatter_3d(df_check, x='ref', y='Right', z='ratio', color='description', symbol = 'ref')
fig = px.scatter_3d(df_check, x='ref', y='ratio', z='Right', color='description')
fig.show()

In [ ]:
#predict
train = complete_data[complete_data['Photometric Interpretation']=='MONOCHROME1'].reset_index()

#print('Id              Actual                 PredictionString')

#for i in range(train['path'].shape[0]):

actual = []
pred = []
Patient_ID = []

cnt = 0
correct = 0

for i in range(50):
    
    status = 0
    #i= 1
   # Patient_ID.append[train['Patient ID'].iloc[i]]

    if train['Photometric Interpretation'].iloc[i]=='MONOCHROME1':        

        #actual 
        if train['Negative for Pneumonia'].iloc[i]==1:
            status = 'negative 1 0 0 1 1'                   
        elif train['Typical Appearance'].iloc[i]==1:
            status = 'typical 1 0 0 1 1'
        elif train['Indeterminate Appearance'].iloc[i]==1:
            status = 'indeterminate 1 0 0 1 1'
        elif train['Atypical Appearance'].iloc[i]==1:
            status = 'atypical 1 0 0 1 1'
        else:
            status = 'none 1 0 0 1 1'        
        
        #prediction

        count_L = []
        count_R = []
        data = []
        img_path = train['path'].iloc[i]
        imgC = read_xray(img_path)   #change from 16bit color to 8bit color
        img_gray = cv2.cvtColor(imgC, cv2.COLOR_BGR2RGB)   # make Gray color
        re_img = cv2.resize(img_gray,(1200,1000))
        hsv = cv2.cvtColor(re_img, cv2.COLOR_BGR2HSV)
        chkGray1(img, hsv, re_img, re_img, train['Patient ID'], data, count_L, count_R)

        if ( (5 < count_R[4] < 19 and 0 < (count_L[4]/count_L[4]) < 10) or (5 < count_R[5] < 19 and 0 < (count_L[4]/count_L[4]) < 10)
           ):
            predict = 'atypical 1 0 0 1 1'

        #elif: 
         #   predict = 'typical 1 0 0 1 1'

        #elif:
        #    predict = 'indeterminate 1 0 0 1 1'
        
       # elif:
        #    predict = 'negative 1 0 0 1 1'

        else:
            predict = 'none 1 0 0 1 1'
            
          

        #print(train['Patient ID'].iloc[i] ,'  ', status,'       ', predict )
        Patient_ID.append(train['Patient ID'].iloc[i])
        actual.append(status)
        pred.append(predict)


In [ ]:
Pred_judg = pd.DataFrame({'Patient_ID':Patient_ID,
                         'actual': actual,
                         'predict': pred })
Pred_judg.head(50)

## MONOCROME2 

In [ ]:
print('Negative_Pneumonia2_cr: ',Negative_Pneumonia2_cr['path'].shape[0], 'images')
print('Typical_Appearance2_cr: ',Typical_Appearance2_cr['path'].shape[0], 'images')
print('Indeterminate_Appearance2_cr: ',Indeterminate_Appearance2_cr['path'].shape[0], 'images')
print('Atypical_Appearance2_cr: ',Atypical_Appearance2_cr['path'].shape[0], 'images')